In [0]:
# Importing libraries
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, LSTM, BatchNormalization, Input
from keras.layers.core import Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import keras

from keras.regularizers import l1, l2, l1_l2
from sklearn.model_selection import train_test_split
from keras.models import load_model 
from sklearn.metrics import accuracy_score

from prettytable import PrettyTable

In [0]:
pt = PrettyTable()
pt.field_names = ['Model', 'Loss', 'Test Accuracy']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

#### Data

In [0]:
# Data directory
DATADIR = 'drive/My Drive/CoLab/Human Activity Recognition/UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/'.join([DATADIR,'{0}/Inertial Signals/{1}_{0}.txt'.format(subset, signal)])
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/'.join([DATADIR,'{0}/y_{0}.txt'.format(subset)])
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [12]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

n_hidden = 32
np.random.seed(23)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

#### Model - 1 Using multilayer LSTM

* Splitting the data into Train and Validation data 

In [0]:
trainX, valX, trainy, valy = train_test_split(X_train, Y_train, test_size=.33, random_state=23)

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(CuDNNLSTM(64, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(Dropout(rate=0.7))

model.add(CuDNNLSTM(32, input_shape=(timesteps, input_dim)))
model.add(Dropout(rate=0.5))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(rate=0.7))

model.add(Dense(n_classes, activation='sigmoid'))

# model.summary()

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
# Training the model
model.fit(trainX,
          trainy,
          batch_size=8,
          validation_data=(valX, valy),
          epochs=50,
          callbacks=[EarlyStopping(monitor='val_acc', patience=25), ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_lr=.0001)])

Train on 4925 samples, validate on 2427 samples
Epoch 1/50
4925/4925 [==============================] - 17s 3ms/step - loss: 1.2205 - acc: 0.4721 - val_loss: 0.9024 - val_acc: 0.4800
Epoch 2/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.8310 - acc: 0.5949 - val_loss: 0.6706 - val_acc: 0.6551
Epoch 3/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.7441 - acc: 0.6311 - val_loss: 0.6190 - val_acc: 0.6593
Epoch 4/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.6962 - acc: 0.6499 - val_loss: 0.6139 - val_acc: 0.6642
Epoch 5/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.7006 - acc: 0.6493 - val_loss: 0.6304 - val_acc: 0.6708
Epoch 6/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.7043 - acc: 0.6491 - val_loss: 0.6064 - val_acc: 0.6576
Epoch 7/50
4925/4925 [==============================] - 10s 2ms/step - loss: 0.6802 - acc: 0.6502 - val_loss: 0.6152 - val_acc: 0.6716
Epoch 8

In [0]:
model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 263us/step


[0.42299219827817475, 0.9002375296912114]

In [0]:
print('Loss : {} | Accuracy : {} %'.format(*np.round([0.42299219827817475, 0.9002375296912114*100.], 3)))

Loss : 0.423 | Accuracy : 90.024 %


* With a simple 2 LSTM layers architecture we got **90.024% test accuracy** and a **loss of 0.423**

In [0]:
pt.add_row(['2 Layers LSTM', 0.423, '90.024 %'])

In [0]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,514,0,1,0,0,22
SITTING,0,394,94,0,0,3
STANDING,0,100,430,2,0,0
WALKING,0,0,0,488,0,8
WALKING_DOWNSTAIRS,0,0,0,1,418,1
WALKING_UPSTAIRS,0,0,0,43,19,409


#### Model - 2 Using Conv nets

Tried with various hyper parameter values by hit and trail.

In [0]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(.001), input_shape=(timesteps, input_dim)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(.0001)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.7))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(.001)))
model.add(BatchNormalization())
model.add(Dropout(0.7))
model.add(Dense(6, activation='softmax'))
# model.summary()

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
!mkdir ./bestModel

In [0]:
!rm ./bestModel/*

In [0]:
# Training the model
model.fit(trainX,
          trainy,
          batch_size=8,
          validation_data=(valX, valy),
          epochs=50,
          callbacks=[ModelCheckpoint('bestModel/bestmodel.hdf5', monitor='val_acc', save_best_only=True),\
                     EarlyStopping(monitor='val_acc', patience=15),\
                     ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_lr=.0001)])

Train on 4925 samples, validate on 2427 samples
Epoch 1/50
4925/4925 [==============================] - 5s 926us/step - loss: 0.2105 - acc: 0.9511 - val_loss: 0.1696 - val_acc: 0.9629
Epoch 2/50
4925/4925 [==============================] - 4s 899us/step - loss: 0.2159 - acc: 0.9486 - val_loss: 0.1637 - val_acc: 0.9633
Epoch 3/50
4925/4925 [==============================] - 4s 896us/step - loss: 0.2086 - acc: 0.9535 - val_loss: 0.1631 - val_acc: 0.9621
Epoch 4/50
4925/4925 [==============================] - 4s 878us/step - loss: 0.2033 - acc: 0.9551 - val_loss: 0.1629 - val_acc: 0.9637
Epoch 5/50
4925/4925 [==============================] - 4s 889us/step - loss: 0.2037 - acc: 0.9539 - val_loss: 0.1610 - val_acc: 0.9633
Epoch 6/50
4925/4925 [==============================] - 4s 903us/step - loss: 0.2063 - acc: 0.9507 - val_loss: 0.1590 - val_acc: 0.9633
Epoch 7/50
4925/4925 [==============================] - 4s 881us/step - loss: 0.2016 - acc: 0.9521 - val_loss: 0.1566 - val_acc: 0.9642


**Loading best model so far**

In [0]:
#Loading best model at epoch with high val accuracy
model = load_model('bestModel/bestmodel.hdf5')

In [0]:
model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 0s 80us/step
[0.343903040034936, 0.9229725144214456]


In [0]:
print('Loss : {} | Accuracy : {} %'.format(*np.round([0.343903040034936, 0.9229725144214456*100.], 2)))

Loss : 0.344 | Accuracy : 92.297 %


* With a simple 2 convolution layers and 1 max-pooling architecture we got **92.297% test accuracy** and a **loss of 0.344**

In [0]:
pt.add_row(['2 Layers 1D Convolutions', 0.344, '92.297 %'])

In [0]:
# Confusion Matrix
confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,512,0,0,0,0,25
SITTING,0,399,68,0,0,24
STANDING,0,72,456,0,0,4
WALKING,0,0,0,481,14,1
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,2,20,449


#### Model - 3 Using Divide and conquer Conv nets

References: 
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5949027/
* https://github.com/maxpumperla/hyperas

##### Data Preparation Basic

In [0]:
from keras.layers import multiply
import keras
import keras.backend as K

In [0]:
def load_y_new(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/'.join([DATADIR,'{0}/y_{0}.txt'.format(subset)])
    y = _read_csv(filename)[0]

    return y.values

In [0]:
X_train, X_test = load_signals('train'), load_signals('test')
y_train, y_test = load_y_new('train'), load_y_new('test')

In [0]:
y_train_bin, y_test_bin = pd.Series(y_train).map(dict(zip(range(1,7), [1]*3+[0]*3))).values,\
                          pd.Series(y_test).map(dict(zip(range(1,7), [1]*3+[0]*3))).values

In [0]:
# Dynamic class data
X_train_dynamic, X_test_dynamic = X_train[y_train_bin==1], X_test[y_test_bin==1]

y_train_dynamic, y_test_dynamic = y_train[y_train_bin==1], y_test[y_test_bin==1] 

# Static class data
X_train_static, X_test_static = X_train[y_train_bin==0], X_test[y_test_bin==0]

y_train_static, y_test_static = y_train[y_train_bin==0], y_test[y_test_bin==0] 

In [0]:
y_train_bin, y_test_bin = pd.get_dummies(y_train_bin).values,\
                                     pd.get_dummies(y_test_bin).values

y_train_dynamic, y_test_dynamic = pd.get_dummies(y_train_dynamic).values,\
                                                 pd.get_dummies(y_test_dynamic).values

y_train_static, y_test_static = pd.get_dummies(y_train_static).values,\
                                              pd.get_dummies(y_test_static).values

##### Base Binary Model

In [0]:
main_input = Input(shape=(timesteps, input_dim), name='main_input')

# Base Binary Model
x = Conv1D(filters=16, kernel_size=3, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(.0001))(main_input)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dropout(rate=.5)(x)
x = Dense(16, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(.001))(x)
x = BatchNormalization()(x)
x = Dropout(rate=.65)(x)
out = Dense(2, activation='softmax', name='binary_out')(x)

bin_model = Model(inputs=main_input, outputs=out)

bin_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(.01), metrics=['accuracy'])

In [0]:
bin_model.fit(X_train,
              y_train_bin,
              batch_size=8,
              validation_data=(X_test, y_test_bin),
              epochs=20,
              callbacks=[EarlyStopping(monitor='val_acc', patience=10), ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, min_lr=.0001)])

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 15s 2ms/step - loss: 0.2863 - acc: 0.9479 - val_loss: 0.1700 - val_acc: 0.9986
Epoch 2/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.2714 - acc: 0.9686 - val_loss: 0.1841 - val_acc: 0.9854
Epoch 3/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.2654 - acc: 0.9767 - val_loss: 0.1852 - val_acc: 0.9946
Epoch 4/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.2800 - acc: 0.9720 - val_loss: 0.1329 - val_acc: 0.9986
Epoch 5/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1795 - acc: 0.9789 - val_loss: 0.0813 - val_acc: 0.9986
Epoch 6/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1202 - acc: 0.9839 - val_loss: 0.0524 - val_acc: 0.9993
Epoch 7/20
7352/7352 [==============================] - 8s 1ms/step - loss: 0.0806 - acc: 0.9902 - val_loss: 0.0423 - val_acc: 0.9986
Epoch 8/20
73

In [0]:
bin_loss, bin_acc = bin_model.evaluate(X_test, y_test_bin)

2947/2947 [==============================] - 0s 141us/step


In [0]:
print('Loss : {} | Accuracy : {} %'.format(bin_loss, bin_acc*100.))

Loss : 0.027248992813763112 | Accuracy : 99.8982015609094 %


In [0]:
# Utility function to print the confusion matrix
def confusion_matrix_bin(Y_true, Y_pred, ACTIVITIES):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [0]:
confusion_matrix_bin(y_test_bin, bin_model.predict(X_test), {
    0: 'Static',
    1: 'Dynamic',
})

Pred,Dynamic,Static
True,,
Dynamic,1387,0
Static,3,1557


In [0]:
bin_model.save('Base_Binary_model.hdf5')

##### Dynamic class Model

In [0]:
# Model - 1
dynamic_input = Input(shape=(timesteps, input_dim), name='dynamic_input')

y = Conv1D(filters=64, kernel_size=3, activation='relu',)(dynamic_input)
y = Conv1D(filters=32, kernel_size=3, activation='relu',)(y)
y = MaxPooling1D(pool_size=2)(y)
y = Flatten()(y)
y = Dropout(rate=.6)(y)
y = Dense(32, activation='relu', kernel_regularizer=l2(.001))(y)
y = BatchNormalization()(y)
y = Dropout(.6)(y)
dynamic_output = Dense(3, activation='softmax', name='dynamic_out')(y)

dynamic_model = Model(inputs=dynamic_input, outputs=dynamic_output)

dynamic_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.rmsprop(.01), metrics=['accuracy'])

In [0]:
dynamic_model.fit(X_train_dynamic,
              y_train_dynamic,
              batch_size=8,
              validation_data=(X_test_dynamic, y_test_dynamic),
              epochs=50,
              callbacks=[EarlyStopping(monitor='val_acc', patience=25), ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_lr=.001)])

Train on 3285 samples, validate on 1387 samples
Epoch 1/50
3285/3285 [==============================] - 3s 787us/step - loss: 0.1978 - acc: 0.9522 - val_loss: 0.1291 - val_acc: 0.9647
Epoch 2/50
3285/3285 [==============================] - 3s 791us/step - loss: 0.2071 - acc: 0.9482 - val_loss: 0.1693 - val_acc: 0.9603
Epoch 3/50
3285/3285 [==============================] - 3s 791us/step - loss: 0.2372 - acc: 0.9434 - val_loss: 0.1323 - val_acc: 0.9733
Epoch 4/50
3285/3285 [==============================] - 3s 782us/step - loss: 0.2026 - acc: 0.9470 - val_loss: 0.1143 - val_acc: 0.9740
Epoch 5/50
3285/3285 [==============================] - 3s 780us/step - loss: 0.2680 - acc: 0.9376 - val_loss: 0.1669 - val_acc: 0.9625
Epoch 6/50
3285/3285 [==============================] - 3s 809us/step - loss: 0.2039 - acc: 0.9528 - val_loss: 0.1494 - val_acc: 0.9668
Epoch 7/50
3285/3285 [==============================] - 3s 789us/step - loss: 0.2005 - acc: 0.9440 - val_loss: 0.1628 - val_acc: 0.9625


In [0]:
dynamic_loss, dynamic_acc = dynamic_model.evaluate(X_test_dynamic, y_test_dynamic)

1387/1387 [==============================] - 0s 91us/step


In [0]:
print('Loss : {} | Accuracy : {} %'.format(dynamic_loss, dynamic_acc*100.))

Loss : 0.2959768250621533 | Accuracy : 96.39509733237203 %


In [0]:
confusion_matrix_bin(y_test_dynamic, dynamic_model.predict(X_test_dynamic), { 0:'WALKING', 1:'WALKING_UPSTAIRS', 2:'WALKING_DOWNSTAIRS'})

Pred,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,
WALKING,482,5,9
WALKING_DOWNSTAIRS,9,405,6
WALKING_UPSTAIRS,2,24,445


In [0]:
dynamic_model.save('Dynamic_class_model.hdf5')

##### Static class model

In [0]:
# Model - 2
static_input = Input(shape=(timesteps, input_dim), name='static_input')

z = Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(.001))(static_input)
z = Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(.1))(z)
z = Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=l2(.001))(z)
z = MaxPooling1D(pool_size=2)(z)
z = Flatten()(z)
z = Dropout(rate=.6)(z)
z = Dense(32, activation='relu', kernel_regularizer=l2())(z)
z = BatchNormalization()(z)
z = Dropout(.7)(z)
static_output = Dense(3, activation='softmax', name='static_out')(z)

static_model = Model(inputs=static_input, outputs=static_output)

static_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.rmsprop(.001), metrics=['accuracy'])

In [0]:
static_model.fit(X_train_static,
              y_train_static,
              batch_size=8,
              validation_data=(X_test_static, y_test_static),
              epochs=100,
              callbacks=[EarlyStopping(monitor='val_acc', patience=10), ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_lr=.001)])

Train on 4067 samples, validate on 1560 samples
Epoch 1/100
4067/4067 [==============================] - 7s 2ms/step - loss: 0.6775 - acc: 0.8190 - val_loss: 0.6473 - val_acc: 0.8423
Epoch 2/100
4067/4067 [==============================] - 3s 828us/step - loss: 0.6001 - acc: 0.8394 - val_loss: 0.4327 - val_acc: 0.8731
Epoch 3/100
4067/4067 [==============================] - 3s 848us/step - loss: 0.5590 - acc: 0.8591 - val_loss: 0.5350 - val_acc: 0.8397
Epoch 4/100
4067/4067 [==============================] - 3s 832us/step - loss: 0.6069 - acc: 0.8439 - val_loss: 0.4331 - val_acc: 0.8833
Epoch 5/100
4067/4067 [==============================] - 3s 827us/step - loss: 0.5809 - acc: 0.8522 - val_loss: 0.5184 - val_acc: 0.8718
Epoch 6/100
4067/4067 [==============================] - 3s 838us/step - loss: 0.5679 - acc: 0.8596 - val_loss: 0.4533 - val_acc: 0.8750
Epoch 7/100
4067/4067 [==============================] - 3s 830us/step - loss: 0.6576 - acc: 0.8289 - val_loss: 0.4280 - val_acc: 0.

In [0]:
static_loss, static_acc = static_model.evaluate(X_test_static, y_test_static)

1560/1560 [==============================] - 0s 92us/step


In [0]:
print('Loss : {} | Accuracy : {} %'.format(static_loss, static_acc*100.))

Loss : 0.23280242435061016 | Accuracy : 92.37179487179488 %


In [0]:
confusion_matrix_bin(y_test_static, static_model.predict(X_test_static), { 0:'SITTING', 1:'STANDING', 2:'LAYING'})

Pred,LAYING,SITTING,STANDING
True,,,
LAYING,537,0,0
SITTING,0,402,89
STANDING,0,30,502


In [0]:
static_model.save('Static_class_model.hdf5')

##### Final Prediction

In [0]:
t = np.zeros((10), dtype='int')

t[[2,6,8]] = [23,54,9]

t

array([ 0,  0, 23,  0,  0,  0, 54,  0,  9,  0])

In [0]:
from keras.models import load_model
from scipy.ndimage import gaussian_filter

class PredictActivity:
  def __init__(self):
    self.binary_model = None
    self.dynamic_model = None
    self.static_model = None

  def loadModels(self, binModelPath, dynamicModelpath, staticModelPath):
    self.binary_model = load_model(binModelPath)
    self.dynamic_model = load_model(dynamicModelpath)
    self.static_model = load_model(staticModelPath)
  
  def predict(self, X):
    y_bin = np.argmax(self.binary_model.predict(X), axis=1)

    X_dynamic = X[y_bin==1]
    X_static = X[y_bin==0]

    # X_dynamic = X_dynamic + .007*(X_dynamic - gaussian_filter(X_dynamic, sigma=8))
    # X_static = X_static + .007*(X_static - gaussian_filter(X_static, sigma=8))

    y_dynamic = np.argmax(self.dynamic_model.predict(X_dynamic), axis=1)
    y_static = np.argmax(self.static_model.predict(X_static), axis=1)

    y_dynamic = y_dynamic + 1
    y_static = y_static + 4

    output = np.zeros((X.shape[0]), dtype='int')

    output[np.where(y_bin==1)[0]] = y_dynamic

    output[np.where(y_bin==0)[0]] = y_static

    return output

In [0]:
predictactivity = PredictActivity()

predictactivity.loadModels('./Base_Binary_model.hdf5', './Dynamic_class_model.hdf5', './Static_class_model.hdf5')

In [0]:
accuracy_score(y_test, predictactivity.predict(X_test))

0.9416355615880556

In [0]:
print('Accuracy : {} %'.format(round(accuracy_score(y_test, predictactivity.predict(X_test))*100., 1)))

Accuracy : 94.2 %


In [0]:
pt.add_row(['Divide and Conquer(CNNs)', '-', '94.2 %'])

In [100]:
print(pt)

+--------------------------+-------+---------------+
|          Model           |  Loss | Test Accuracy |
+--------------------------+-------+---------------+
|      2 Layers LSTM       | 0.423 |    90.024 %   |
| 2 Layers 1D Convolutions | 0.344 |    92.297 %   |
| Divide and Conquer(CNNs) |   -   |     94.2 %    |
+--------------------------+-------+---------------+
